![Arango Logo](https://github.com/QubitPi/QubitPi/blob/master/img/arango-logo-with-text.png?raw=true "Error loading QubitPi/img/arango-logo-with-text.png")

Arango AQL Tutorial
===================

This is a tutorial on ArangoDB’s query language AQL, built around a small dataset of characters from the novel and fantasy drama television series Game of Thrones (as of season 1). It includes character traits in two languages, some family relations, and last but not least a small set of filming locations, which makes for an interesting mix of data to work with.

Setup
-----

In [1]:
import json
import requests
import sys
import oasis # this is a local module "notebooks/arangodb/oasis.py"
import time

from pyArango.connection import *
from arango import ArangoClient

Let's Create the temporary database:

*Note: this notebook uses a temporary instance which will be autodeleted!*

In [2]:
login = oasis.getTempCredentials(
    tutorialName="AQLCrudTutorial",
    credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB'
)

database = oasis.connect_python_arango(login)
aql = database.aql

print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Reusing cached credentials.
https://tutorials.arangodb.cloud:8529
Username: TUT1cihi64f6xwx1p0jfghlo
Password: TUT57a9pqx8wd466m36fm69hu
Database: TUTq8tm1n269ba2h54k0fr26


We should be able to see an URL, a Username, a Password, and a Database printed above. Feel free to click the URL which will take us to the ArangoDB UI. On the login page, use the Username and Password above as thelogin credential, then on the next page choose the printed Database as the selected database. The UI should look like the screenshot below:

![Arango UI](https://github.com/QubitPi/QubitPi/blob/master/img/arango-ui.png?raw=true "Error loading QubitPi/img/arango-ui.png")

Creating Collections
--------------------

Before we can insert documents (data) with AQL into database, we need a place to put them in - a __Collection__. Collections can be managed via the web interface, arangosh or a driver. It is not possible to do so with AQL however.

In [3]:
if not database.has_collection("Characters"):
  database.create_collection("Characters")

print("We have 'Characters' collection now.")

We have 'Characters' collection now.


Creating and Reading Documents
------------------------------

The syntax for creating a new [document](https://arango.qubitpi.org/stable/concepts/data-structure/#documents) is `INSERT document INTO collectionName`. The document is an object like we may know it from JavaScript or JSON, which is comprised of attribute key and value pairs. The quotes around the attribute keys are optional in AQL. Keys are always character sequences (strings), whereas attribute values can have different types:

- null
- boolean (true, false)
- number (integer and floating point)
- string
- array
- object

Name and surname of the character document we will be inserting are both string values. The alive state uses a boolean. Age is a numeric value. The traits are an array of strings. The entire document is an object.

In [4]:
insert_query = """
    INSERT {
        "name": "Ned",
        "surname": "Stark",
        "alive": true,
        "age": 41,
        "traits": ["A","H","C","N","P"]
    } INTO Characters
    """

aql.execute(insert_query)

<Cursor>

We can also execute all the queries on the ArangoDB Web UI:

![](./img/aql-on-ui.png)

Let's check whether the insert was sucessfull by querying the `Characters` collections. The syntax of the loop is `FOR variableName IN collectionName`.

In [5]:
all_characters = """
    FOR character IN Characters
        RETURN character
    """

query_result = aql.execute(all_characters)

for doc in query_result:
    print(doc)
    print()

{'_key': '266270034987', '_id': 'Characters/266270034987', '_rev': '_i4UDWxm---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



For each document in the collection, `character` is assigned a document, which is then returned as per the loop body.

Each document features the 5 attributes we stored, plus 3 more added by the database system

1. a unique `_key`, which identifies it within a collection
2. an `_id` which is a computed property - a concatenation of the collection name, a forward slash `/` and the document key. It uniquely identies a document within a database
3. `_rev`, the revision ID managed by the system. Older revisions of a document cannot be accessed.

Document keys can be provided by the user upon document creation, or a unique value is assigned automatically. It can not be changed later. All 3 system attributes starting with an underscore `_` are read-only.

Next, let us add some more characters. We use the `LET` keyword to define a named variable with an array of objects as value, so `LET variableName = valueExpression` and the expression being a literal array definition like `[ {...}, {...}, ... ]`.

`FOR variableName IN expression` is used to iterate over each element of the data array. In each loop, one element is assigned to the variable `d`. This variable is then used in the `INSERT` statement instead of a literal object definition. What is does is basically:

In [6]:
insert_query = """
    LET data = [
        { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
        { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
        { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
        { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
        { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
        { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
        { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
        { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
        { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
        { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
        { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
        { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
        { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
        { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
        { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
        { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
        { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
        { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
        { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
        { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
        { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
        { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
        { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
        { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
        { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
        { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
        { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
        { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
        { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
        { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
        { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
        { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
        { "name": "Gilly", "alive": true, "traits": ["L","J"] },
        { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
        { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
        { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
        { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
        { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
        { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
        { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
        { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
        { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
    ]
    
    FOR d IN data
        INSERT d INTO Characters
    """

aql.execute(insert_query)

<Cursor>

As before let's check the `Characters` collection, but this time only return each characters name:

In [7]:
all_characters_names = """
    FOR character IN Characters
        RETURN character.name
    """

query_result = aql.execute(all_characters_names)

for doc in query_result:
    print(doc)
    print()

Ned

Robert

Jaime

Catelyn

Cersei

Daenerys

Jorah

Petyr

Viserys

Jon

Sansa

Arya

Robb

Theon

Bran

Joffrey

Sandor

Tyrion

Khal

Tywin

Davos

Samwell

Stannis

Melisandre

Margaery

Jeor

Bronn

Varys

Shae

Talisa

Gendry

Ygritte

Tormund

Gilly

Brienne

Ramsay

Ellaria

Daario

Missandei

Tommen

Jaqen

Roose

The High Sparrow



Updating Documents
------------------

Let's say we need to change the alive attribute of Ned. For this we first identify the `_key` attribute of Ned

In [8]:
find_ned_query = """
    FOR character IN Characters
        FILTER character.name == "Ned"
        RETURN character._key
    """

neds_document_key = None

query_result = aql.execute(find_ned_query)

for doc in  query_result:
    print("_key: " + str(doc))
    neds_document_key = doc
    print()

_key: 266270034987



Using `key` we can update an existing document:

In [9]:
kill_ned = """
    UPDATE @key 
    WITH { alive: false} 
    IN Characters
"""
bindVars = {'key': neds_document_key}
aql.execute(kill_ned, batch_size=1, bind_vars=bindVars)

find_ned_query = """
    FOR character IN Characters
        FILTER character.name == "Ned"
        RETURN character
"""
query_result = aql.execute(find_ned_query)

for doc in query_result:
    print(doc)
    print()

{'_key': '266270034987', '_id': 'Characters/266270034987', '_rev': '_i4UDZYC---', 'name': 'Ned', 'surname': 'Stark', 'alive': False, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



We could have also replaced the entire document content, using `REPLACE` instead of `UPDATE`:

In [10]:
kill_ned = """
    REPLACE @key WITH {
        name: "Ned",
        surname: "Stark",
        alive: false,
        age: 41,
        traits: ["A","H","C","N","P"]
    } IN Characters
"""
bindVars = {'key': neds_document_key}
aql.execute(kill_ned, batch_size=1, bind_vars=bindVars)

find_ned_query = """
FOR character IN Characters
    FILTER character.name == "Ned"
    RETURN character
"""
query_result = aql.execute(find_ned_query)

for doc in query_result:
    print(doc)
    print()

{'_key': '266270034987', '_id': 'Characters/266270034987', '_rev': '_i4UDaUa--_', 'age': 41, 'alive': False, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P']}



We could again use the `FOR` loop construct from before to update all characters:

In [11]:
season_query = """
    FOR character IN Characters
    UPDATE character WITH { season: 1 } IN Characters
"""
aql.execute(season_query)

all_characters_names_season = """
    FOR character IN Characters
    RETURN {"Name" : character.name, "Season" : character.season}
"""
query_result = aql.execute(all_characters_names_season)

for doc in query_result:
    print(doc)
    print()

{'Name': 'Ned', 'Season': 1}

{'Name': 'Robert', 'Season': 1}

{'Name': 'Jaime', 'Season': 1}

{'Name': 'Catelyn', 'Season': 1}

{'Name': 'Cersei', 'Season': 1}

{'Name': 'Daenerys', 'Season': 1}

{'Name': 'Jorah', 'Season': 1}

{'Name': 'Petyr', 'Season': 1}

{'Name': 'Viserys', 'Season': 1}

{'Name': 'Jon', 'Season': 1}

{'Name': 'Sansa', 'Season': 1}

{'Name': 'Arya', 'Season': 1}

{'Name': 'Robb', 'Season': 1}

{'Name': 'Theon', 'Season': 1}

{'Name': 'Bran', 'Season': 1}

{'Name': 'Joffrey', 'Season': 1}

{'Name': 'Sandor', 'Season': 1}

{'Name': 'Tyrion', 'Season': 1}

{'Name': 'Khal', 'Season': 1}

{'Name': 'Tywin', 'Season': 1}

{'Name': 'Davos', 'Season': 1}

{'Name': 'Samwell', 'Season': 1}

{'Name': 'Stannis', 'Season': 1}

{'Name': 'Melisandre', 'Season': 1}

{'Name': 'Margaery', 'Season': 1}

{'Name': 'Jeor', 'Season': 1}

{'Name': 'Bronn', 'Season': 1}

{'Name': 'Varys', 'Season': 1}

{'Name': 'Shae', 'Season': 1}

{'Name': 'Talisa', 'Season': 1}

{'Name': 'Gendry', 'Seas

Note, that here we customized the `RETURN` to return a json document consisting of name and season.

Filtering Documents
-------------------

To find documents that fulfill certain criteria more complex than key equality, there is the `FILTER` operation in AQL, which enables us to formulate arbitrary conditions for documents to match.

We actually have used a filter condition before to find `Ned`:

In [12]:
find_ned_query = """
    FOR character IN Characters
        FILTER character.name == "Ned"
        RETURN character
"""
query_result = aql.execute(find_ned_query)
for doc in query_result:
    print(doc)
    print()

{'_key': '266270034987', '_id': 'Characters/266270034987', '_rev': '_i4UDbP6---', 'age': 41, 'alive': False, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P'], 'season': 1}



The filter condition reads like: “the attribute name of a character document must be equal to the string Ned”. If the condition applies, character document gets returned. This works with any attribute likewise:

In [13]:
find_ned_query = """
    FOR character IN Characters
        FILTER character.surname == "Stark"
        RETURN character
"""
query_result = aql.execute(find_ned_query)
for doc in query_result:
    print(doc)
    print()

{'_key': '266270034987', '_id': 'Characters/266270034987', '_rev': '_i4UDbP6---', 'age': 41, 'alive': False, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P'], 'season': 1}

{'_key': '266270034991', '_id': 'Characters/266270034991', '_rev': '_i4UDbP6--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'traits': ['D', 'H', 'C'], 'season': 1}

{'_key': '266270034998', '_id': 'Characters/266270034998', '_rev': '_i4UDbP6--I', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'traits': ['D', 'I', 'J'], 'season': 1}

{'_key': '266270034999', '_id': 'Characters/266270034999', '_rev': '_i4UDbP6--J', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'traits': ['C', 'K', 'L'], 'season': 1}

{'_key': '266270035000', '_id': 'Characters/266270035000', '_rev': '_i4UDbP6--K', 'name': 'Robb', 'surname': 'Stark', 'alive': False, 'traits': ['A', 'B', 'C', 'K'], 'season': 1}

{'_key': '266270035002', '_id': 'Characters/266270035002', '_rev'

### Range Conditions

Strict equality is one possible condition we can state. There are plenty of other conditions we can formulate however. For example, we could ask for all young characters:

In [14]:
find_adults_query = """
    FOR character IN Characters
        FILTER character.age >= 13
        RETURN character.name
"""
query_result = aql.execute(find_adults_query)

for doc in query_result:
    print(doc)
    print()

Ned

Jaime

Catelyn

Cersei

Daenerys

Jon

Sansa

Theon

Joffrey

Tyrion

Davos

Samwell

Brienne



The operator `>=` stands for greater-or-equal, so every character of age 13 or older is returned (only their name in the example). We can return names and age of all characters younger than 13 by changing the operator to less-than and using the object syntax to define a subset of attributes to return:

In [15]:
find_young_query = """
    FOR character IN Characters
        FILTER character.age < 13
        RETURN { name: character.name, age: character.age }
"""
query_result = aql.execute(find_young_query)
for doc in query_result:
    print(doc)
    print()   

{'name': 'Robert', 'age': None}

{'name': 'Jorah', 'age': None}

{'name': 'Petyr', 'age': None}

{'name': 'Viserys', 'age': None}

{'name': 'Arya', 'age': 11}

{'name': 'Robb', 'age': None}

{'name': 'Bran', 'age': 10}

{'name': 'Sandor', 'age': None}

{'name': 'Khal', 'age': None}

{'name': 'Tywin', 'age': None}

{'name': 'Stannis', 'age': None}

{'name': 'Melisandre', 'age': None}

{'name': 'Margaery', 'age': None}

{'name': 'Jeor', 'age': None}

{'name': 'Bronn', 'age': None}

{'name': 'Varys', 'age': None}

{'name': 'Shae', 'age': None}

{'name': 'Talisa', 'age': None}

{'name': 'Gendry', 'age': None}

{'name': 'Ygritte', 'age': None}

{'name': 'Tormund', 'age': None}

{'name': 'Gilly', 'age': None}

{'name': 'Ramsay', 'age': None}

{'name': 'Ellaria', 'age': None}

{'name': 'Daario', 'age': None}

{'name': 'Missandei', 'age': None}

{'name': 'Tommen', 'age': None}

{'name': 'Jaqen', 'age': None}

{'name': 'Roose', 'age': None}

{'name': 'The High Sparrow', 'age': None}



We may notice that it returns name and age of 30 characters, most with an age of `null`. The reason is that `null` is the fallback value if an attribute is requested by the query, but no such attribute exists in the document, and the `null` is compares to numbers as lower (see [Type and value order](https://arango.qubitpi.org/stable/aql/fundamentals/type-and-value-order/)). Hence, it accidentally fulfills the age criterion `character.age < 13` (`null < 13`). To not let documents pass the filter without an age attribute, we can add a second criterion:

In [16]:
find_young_query = """
    FOR character IN Characters
        FILTER character.age < 13
        FILTER character.age != null
        RETURN { name: character.name, age: character.age }
"""
query_result = aql.execute(find_young_query)
for doc in query_result:
    print(doc)
    print()   

{'name': 'Arya', 'age': 11}

{'name': 'Bran', 'age': 10}



This could equally be written with a boolean `AND` operator as:

In [17]:
find_young_query = """
    FOR character IN Characters
        FILTER character.age < 13 AND character.age != null
        RETURN { name: character.name, age: character.age }
"""
query_result = aql.execute(find_young_query)
for doc in query_result:
    print(doc)
    print()  

{'name': 'Arya', 'age': 11}

{'name': 'Bran', 'age': 10}



If we want documents to fulfill one or another condition, possibly for different attributes as well, use `OR`:

In [18]:
find_joffrey_query = """
    FOR character IN Characters
        FILTER character.name == "Jon" OR character.name == "Joffrey"
        RETURN { name: character.name, surname: character.surname }
"""
query_result = aql.execute(find_joffrey_query)
for doc in query_result:
    print(doc)
    print()  

{'name': 'Jon', 'surname': 'Snow'}

{'name': 'Joffrey', 'surname': 'Baratheon'}



To learn more about Filter Operation check the [documentation](https://arango.qubitpi.org/stable/aql/high-level-operations/filter/).

Sorting Documents
-----------------

To return characters in a defined order, we can add a `SORT()` operation.

In [19]:
all_characters_names = """
    FOR character IN Characters
        SORT character.name
        RETURN character.name
"""
query_result = aql.execute(all_characters_names)
for doc in query_result:
    print(doc)
    print()

Arya

Bran

Brienne

Bronn

Catelyn

Cersei

Daario

Daenerys

Davos

Ellaria

Gendry

Gilly

Jaime

Jaqen

Jeor

Joffrey

Jon

Jorah

Khal

Margaery

Melisandre

Missandei

Ned

Petyr

Ramsay

Robb

Robert

Roose

Samwell

Sandor

Sansa

Shae

Stannis

Talisa

The High Sparrow

Theon

Tommen

Tormund

Tyrion

Tywin

Varys

Viserys

Ygritte



We can reverse the sort order with `DESC`:

In [20]:
all_characters_names = """
    FOR character IN Characters
        SORT character.name DESC
        RETURN character.name
"""
query_result = aql.execute(all_characters_names)
for doc in query_result:
    print(doc)
    print()

Ygritte

Viserys

Varys

Tywin

Tyrion

Tormund

Tommen

Theon

The High Sparrow

Talisa

Stannis

Shae

Sansa

Sandor

Samwell

Roose

Robert

Robb

Ramsay

Petyr

Ned

Missandei

Melisandre

Margaery

Khal

Jorah

Jon

Joffrey

Jeor

Jaqen

Jaime

Gilly

Gendry

Ellaria

Davos

Daenerys

Daario

Cersei

Catelyn

Bronn

Brienne

Bran

Arya



### Sorting by Multiple Attributes

Assume we want to sort by surname. Many of the characters share a surname. The result order among characters with the same surname is then undefined. We can first sort by surname, then name to determine the order:

In [21]:
all_characters_names = """
    FOR character IN Characters
        FILTER character.surname
        SORT character.surname, character.name
        LIMIT 10
        RETURN {
            surname: character.surname,
            name: character.name
        }
"""

query_result = aql.execute(all_characters_names)
for doc in query_result:
    print(doc)
    print()

{'surname': 'Baelish', 'name': 'Petyr'}

{'surname': 'Baratheon', 'name': 'Joffrey'}

{'surname': 'Baratheon', 'name': 'Robert'}

{'surname': 'Baratheon', 'name': 'Stannis'}

{'surname': 'Baratheon', 'name': 'Tommen'}

{'surname': 'Bolton', 'name': 'Ramsay'}

{'surname': 'Bolton', 'name': 'Roose'}

{'surname': 'Clegane', 'name': 'Sandor'}

{'surname': 'Drogo', 'name': 'Khal'}

{'surname': 'Giantsbane', 'name': 'Tormund'}



Overall, the documents are sorted by last name. If the surname is the same for two characters, the name values are compared and the result sorted.

More information about `SORT` can be found in its [documentation](https://arango.qubitpi.org/stable/aql/high-level-operations/sort/)

Pagination
----------

It may not always be necessary to return all documents, that a `FOR` loop would normally return. In those cases, we can limit the amount of documents with a `LIMIT()` operation:

In [22]:
sample_chars_query = """
    FOR character IN Characters
        SORT character.name
        LIMIT 5
        RETURN character.name
"""

query_result = aql.execute(sample_chars_query)
for doc in query_result:
    print(doc)
    print()  

Arya

Bran

Brienne

Bronn

Catelyn



When `LIMIT` is followed 2 numbers, however, the first number specifies the number of documents that are to be skipped and return the next second-number documents. This effectively achieves the pagination:

In [23]:
sample_chars_query = """
FOR character IN Characters
    SORT character.name
    LIMIT 2, 5
    RETURN character.name
"""
query_result = aql.execute(sample_chars_query)
for doc in query_result:
    print(doc)
    print()  

Brienne

Bronn

Catelyn

Cersei

Daario



More information about `LIMIT` can be found in its [documentation](https://arango.qubitpi.org/stable/aql/high-level-operations/limit/)

Putting Everying Together
-------------------------

Let's address a slightly complex business query - finds the 10 youngest characters:

In [24]:
sample_chars_query = """
    FOR character IN Characters
        FILTER character.age
        SORT character.age
        LIMIT 10
        RETURN {
            name: character.name,
            age: character.age
        }
"""

query_result = aql.execute(sample_chars_query)
for doc in query_result:
    print(doc)
    print()  

{'name': 'Bran', 'age': 10}

{'name': 'Arya', 'age': 11}

{'name': 'Sansa', 'age': 13}

{'name': 'Daenerys', 'age': 16}

{'name': 'Jon', 'age': 16}

{'name': 'Theon', 'age': 16}

{'name': 'Samwell', 'age': 17}

{'name': 'Joffrey', 'age': 19}

{'name': 'Tyrion', 'age': 32}

{'name': 'Brienne', 'age': 32}



Deleting Documents
------------------

To fully remove documents from a collection, there is the `REMOVE` operation. It works similar to the other modification operations, yet without a `WITH` clause:

In [25]:
remove_ned = """
    REMOVE @key IN Characters
"""
bindVars = {'key': neds_document_key}

try:
    aql.execute(remove_ned, bind_vars=bindVars)
except:
    print("Ned already removed.")

find_ned_query = """
    FOR character IN Characters
    FILTER character.name == "Ned"
    RETURN character
"""
query_result = aql.execute(find_ned_query, count=True)

if len(query_result) == 0 :
    print("Ned not found.")

Ned not found.


As we might have already guessed we can again use a `FOR` loop if we want to perform this operation for the entire collection:

In [26]:
remove_all = """
    FOR character IN Characters
    REMOVE character IN Characters
"""
aql.execute(remove_all)
    
all_characters_names = """
    FOR character IN Characters
    RETURN character
"""
query_result = aql.execute(all_characters_names, count=True)

if len(query_result) == 0 :
    print("No characters left.")

No characters left.
